## Week 6 - Characterizing egg variation

In [6]:
# Load libraries
import duckdb
import pandas as pd
import os

# Connect to the database
db_path = os.path.join(os.getcwd(), 'bren-eds213-data', 'database', 'database.db')
conn = duckdb.connect(db_path)
c = conn.cursor()

# Get distinct species with egg data
species_query = """
SELECT DISTINCT s.Code, s.Scientific_name
FROM Species s
JOIN Bird_nests n ON s.Code = n.Species
JOIN Bird_eggs e ON n.Nest_ID = e.Nest_ID
ORDER BY s.Scientific_name;
"""

# Loop over species
for row in c.execute(species_query).fetchall():
    species_code = row[0]
    scientific_name = row[1]

    # Query egg volume data for the species
    egg_query = """
    SELECT Width * Width * Length AS Volume
    FROM Bird_eggs
    JOIN Bird_nests ON Bird_eggs.Nest_ID = Bird_nests.Nest_ID
    WHERE Bird_nests.Species = ?
    """
    df = pd.read_sql(egg_query, conn, params=(species_code,))

    # Skip species with no valid data
    if df.empty or df['Volume'].mean() == 0:
        continue

    # Compute coefficient of variation (CV) and print
    cv = round(df['Volume'].std() / df['Volume'].mean() * 100, 2)
    print(f"{scientific_name} {cv}%")

Arenaria interpres 21.12%
Calidris alpina 5.46%
Calidris fuscicollis 16.77%
Charadrius semipalmatus 8.99%
Phalaropus fulicarius 4.65%
Pluvialis dominica 19.88%
Pluvialis squatarola 6.94%


/var/folders/xv/0wmfjsw11859qnxbtrgsp26m0000gn/T/ipykernel_19246/3257002509.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(egg_query, conn, params=(species_code,))
